In [2]:
import pandas as pd
import numpy as np
import sys

# Kütüphane kurulumu
!{sys.executable} -m pip install -q scikit-learn

# CSV yolunu ayarla
csv_path = "Police_Transparency_-_Arrests_-_All_Data_(main_table___denormalized).csv"

#Dosya yolu konrtolü 
try:
    # Veriyi yükle
    df = pd.read_csv(csv_path, low_memory=False)
    print("✔ Veri seti başarıyla yüklendi.")
except Exception as e:
    print(f"❌ Veri yükleme hatası: {e}")
    raise SystemExit
    
# Boyut bilgisi
print(f"Veri boyutu: {df.shape}")
display(df.head())


✔ Veri seti başarıyla yüklendi.
Veri boyutu: (47444, 40)


,X,Y,rin,primary_key,arrest_type,arrest_translation,arrest_dt,arrest_time,arrest_hour_of_day,location,...,severity_code,severity_trans,ChargeClassTranslation,ChargeGrouping,arrestee_sex,arrestee_race,arrestee_ethnicity,arrestee_genderTranslation,arrestee_RaceAndEthnicity,ESRI_OID
0,693985,882475,110528,TE20226631,O,On-View: Arrested when first observed/investi...,2022/09/24 21:17:00+00,2117,21,2XX E 5TH ST,...,NaN,Unknown,Arizona Revised Statutes,Miscellaneous,M,B,N,Male,Black or African American,3715748
1,710663,880237,110739,TE20226842,T,Taken Into Custody: Arrest on warrant or PC f...,2022/10/04 17:30:00+00,1730,17,9XX S ACORN AVE,...,F,Felony,Arizona Revised Statutes,Drug charges,M,W,N,Male,White,3715749
2,704259,878441,110587,TE20226690,T,Taken Into Custody: Arrest on warrant or PC f...,2022/09/27 21:52:00+00,2152,21,1XXX E APACHE BLVD,...,M,Misdemeanor,Arizona Revised Statutes,Assault & related charges,M,W,H,Male,Hispanic or Latino,3715750
3,693160,868321,110521,TE20226624,O,On-View: Arrested when first observed/investi...,2022/09/24 21:00:00+00,2100,21,4XXX S MILL AVE,...,F,Felony,Arizona Revised Statutes,Miscellaneous,M,B,N,Male,Black or African American,3715751
4,708569,883842,110789,TE20226892,T,Taken Into Custody: Arrest on warrant or PC f...,2022/10/07 03:54:00+00,354,3,2XXX W RIO SALADO PKWY,...,M,Misdemeanor,Arizona Revised Statutes,Escape & related charges,M,W,H,Male,Hispanic or Latino,3715752


In [3]:
print("\n--- TEKİLLEŞTİRME ---")

initial_rows = df.shape[0]

# Yinelenen satırları sil
df.drop_duplicates(inplace=True)

# Silinen satır sayısını hesapla
removed = initial_rows - df.shape[0]

print(f"✔ {removed} adet yinelenen satır silindi.")
print(f"Yeni veri boyutu: {df.shape}")



--- TEKİLLEŞTİRME ---
✔ 0 adet yinelenen satır silindi.
Yeni veri boyutu: (47444, 40)


In [4]:
# Silinecek sütunların listesi
columns_to_drop = [
    'X','Y','rin','primary_key','charge_rin','pin','ESRI_OID',
    'x_coordinate','y_coordinate',
    'arrest_officer',
    'ofc_age_range','ofc_genderTranslation','ofc_RaceAndEthnicity',
    'statute','class','severity_code'
]

# HATA ÖNLEYİCİ: Veri setinde gerçekten var olan sütunları seç
existing_cols = [col for col in columns_to_drop if col in df.columns]
df.drop(columns=existing_cols, inplace=True)

print(f"✔ {len(existing_cols)} sütun kaldırıldı.")
print("Kaldırılanlar:", existing_cols)


✔ 16 sütun kaldırıldı.
Kaldırılanlar: ['X', 'Y', 'rin', 'primary_key', 'charge_rin', 'pin', 'ESRI_OID', 'x_coordinate', 'y_coordinate', 'arrest_officer', 'ofc_age_range', 'ofc_genderTranslation', 'ofc_RaceAndEthnicity', 'statute', 'class', 'severity_code']


In [5]:
print("\n--- Eksik Veri Analizi ---")

# Her sütundaki eksik veri oranını (%) hesapla ve büyükten küçüğe sırala
missing = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)

# En çok eksik veriye sahip ilk 20 sütunu göster
display(missing.head(20))
print("\nVeri boyutu:", df.shape)



--- Eksik Veri Analizi ---


area_name                     5.667735
zipcode                       5.225107
arrest_translation            1.199309
arrest_type                   1.199309
ChargeGrouping                0.265576
arrestee_ethnicity            0.073771
arrestee_age_range            0.067448
arrestee_RaceAndEthnicity     0.040047
arrestee_genderTranslation    0.040047
arrestee_race                 0.040047
arrestee_sex                  0.040047
charge_count                  0.002108
ChargeClassTranslation        0.000000
severity_trans                0.000000
arrest_hour_of_day            0.000000
location                      0.000000
arrest_dt                     0.000000
arrest_time                   0.000000
grid                          0.000000
zone                          0.000000
dtype: float64


Veri boyutu: (47444, 24)


In [6]:
print("\n--- Aykırı Değer İşleme ---")
#Boxplot 

if "charge_count" in df.columns:
    # 1. Çeyrekleri (Q1, Q3) ve Aralığı (IQR) hesapla
    Q1 = df['charge_count'].quantile(0.25)
    Q3 = df['charge_count'].quantile(0.75)
    IQR = Q3 - Q1

    # 2. Üst sınır belirle (Standart formül)
    upper = Q3 + 1.5*IQR
 
    # 3. Kaç anomali var?
    anomalies = df[df['charge_count'] > upper].shape[0]

    # 4. Anomalileri silmek yerine 'NaN' yap
    df.loc[df['charge_count'] > upper, 'charge_count'] = np.nan
    print(f"✔ {anomalies} adet aykırı değer NaN yapıldı.")
else:
    print("⚠ 'charge_count' bulunamadı.")



--- Aykırı Değer İşleme ---
✔ 856 adet aykırı değer NaN yapıldı.


In [7]:
# Sadece metin, kategori veya mantıksal (bool) sütunları seç
categorical_cols = df.select_dtypes(include=['object','category','bool']).columns

threshold = 0.001
print(f"\n--- Düşük Frekanslı Kategoriler (%{threshold*100}) ---")

for col in categorical_cols:
    # Kategorilerin görülme sıklığını oran (yüzde) olarak hesapla
    vc = df[col].value_counts(normalize=True)

    # Eşiğin altında kalan "nadir" kategorileri listele
    low = vc[vc < threshold].index.tolist()
    
    if low:
        # Sütun adı, kaç tane nadir veri olduğu ve ilk 3 örneği yazdır
        print(f"• {col}: {len(low)} düşük frekanslı kategori → {low[:3]}...")



--- Düşük Frekanslı Kategoriler (%0.1) ---
• arrest_type: 1 düşük frekanslı kategori → ['S']...
• arrest_translation: 1 düşük frekanslı kategori → ['Summoned/Cited: Ticket issued, or long form charges approved, not taken to jail']...
• arrest_dt: 20549 düşük frekanslı kategori → ['2023/02/11 20:00:00+00', '2024/05/09 16:10:00+00', '2023/06/02 10:14:00+00']...
• location: 2015 düşük frekanslı kategori → ['4XX W BROADWAY RD    ', '1XXX E VISTA DEL CERRO DR     ', ' PRIEST DR / W SOUTHERN AVE  ']...
• municipality: 2 düşük frekanslı kategori → ['SR        ', 'CH        ']...
• district: 2 düşük frekanslı kategori → ['UI    ', 'DT    ']...
• zone: 2 düşük frekanslı kategori → ['UI    ', '1     ']...
• grid: 223 düşük frekanslı kategori → ['2311  ', '0606  ', '1317  ']...
• charge: 5889 düşük frekanslı kategori → ['PRESCRIPT DRUG-POSSESS/USE                                                                                                                                                       

In [8]:
print("\n--- İmputasyon (Eksik Veri Doldurma) ---")

# SAYISAL İMPUTASYON (MOD)
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    if df[col].isna().any():
        # Sütundaki en sık geçen değeri (Mod) bul
        mode_val = df[col].mode(dropna=True)[0]
        
        #NOT: uyarı almamak için inplace kullanılmıyor 
        df[col] = df[col].fillna(mode_val)
        
        print(f"✔ '{col}' mod ({mode_val}) ile dolduruldu.")

# KATEGORİK İMPUTASYON (DAĞILIM KORUMA)
cat_cols = df.select_dtypes(include=['object','category','bool']).columns

def impute_with_dist(col):
    nonnull = df[col].dropna()
    if nonnull.empty:
        return  # tüm kolon NaN ise

    # Mevcut sınıfların görülme oranlarını hesapla
    probs = nonnull.value_counts(normalize=True)
    
    # Sadece boş olan yerleri tespit et
    nan_mask = df[col].isna()
    nan_count = nan_mask.sum()
    if nan_count == 0:
        return
        
    # Hesaplanan oranlara göre rastgele seçim yap
    random_values = np.random.choice(probs.index, size=nan_count, p=probs.values)
  
    df.loc[nan_mask, col] = random_values
    print(f"✔ '{col}' kategorik imputasyon ile dolduruldu.")

for col in cat_cols:
    if df[col].isna().any():
        impute_with_dist(col)

print("\n✔ Tüm imputasyonlar uyarısız olarak tamamlandı.")



--- İmputasyon (Eksik Veri Doldurma) ---
✔ 'zipcode' mod (85281.0) ile dolduruldu.
✔ 'charge_count' mod (1.0) ile dolduruldu.
✔ 'arrest_type' kategorik imputasyon ile dolduruldu.
✔ 'arrest_translation' kategorik imputasyon ile dolduruldu.
✔ 'area_name' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_age_range' kategorik imputasyon ile dolduruldu.
✔ 'ChargeGrouping' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_sex' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_race' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_ethnicity' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_genderTranslation' kategorik imputasyon ile dolduruldu.
✔ 'arrestee_RaceAndEthnicity' kategorik imputasyon ile dolduruldu.

✔ Tüm imputasyonlar uyarısız olarak tamamlandı.


In [9]:
print("\n--- Zaman Feature Engineering ---")

if 'arrest_dt' in df.columns:
    # Karışık metni gerçek tarih formatına (datetime) çevir
    df['arrest_dt'] = pd.to_datetime(df['arrest_dt'].astype(str).str.split().str[0], errors='coerce')

    # Tarihten yeni özellikler türet (yıl, ay, gün)
    df['arrest_year'] = df['arrest_dt'].dt.year
    df['arrest_month'] = df['arrest_dt'].dt.month
    df['arrest_day_of_week'] = df['arrest_dt'].dt.dayofweek

    # Saati günün dilimlerine ayır (binning)
    def get_time_cat(h):
        if pd.isna(h): return 'Bilinmiyor'
        if 0<=h<=6: return 'Gece'
        if 7<=h<=11: return 'Sabah'
        if 12<=h<=16: return 'Ogle'
        if 17<=h<=20: return 'Ikindi'
        return 'Aksam'

    # 'arrest_hour_of_day' sütunu varsa fonksiyonu uygula
    if 'arrest_hour_of_day' in df.columns:
        df['time_category'] = df['arrest_hour_of_day'].apply(get_time_cat)

    # Artık işimize yaramayan ham tarih sütunlarını sil
    df.drop(columns=[c for c in ['arrest_dt','arrest_time'] if c in df.columns], inplace=True)
    print("✔ Zaman özellikleri eklendi.")



--- Zaman Feature Engineering ---
✔ Zaman özellikleri eklendi.


In [10]:
# Hedef değişkeni oluştur
if 'is_onview_arrest' not in df.columns:
    df['is_onview_arrest'] = np.where(df['arrest_type'] == 'O', 1, 0)

# Kaynak sütunu sil (Varsa)
cols_to_drop = [c for c in ['arrest_type', 'arrest_translation'] if c in df.columns]
df.drop(columns=cols_to_drop, inplace=True)

print("✔ Hedef değişken 'is_onview_arrest' oluşturuldu.")

✔ Hedef değişken 'is_onview_arrest' oluşturuldu.


In [11]:
# 50'den az seçeneği olan sütunları seç 
ohe_cols = [c for c in df.select_dtypes(include=['object','category','bool'])
            if df[c].nunique() < 50]

#One-Hot Encoding uygula 
df = pd.get_dummies(df, columns=ohe_cols, drop_first=True, dtype=int)

print(f"✔ OHE tamamlandı. Yeni sütun sayısı: {df.shape[1]}")


✔ OHE tamamlandı. Yeni sütun sayısı: 112


In [12]:
#Farklı birimdeki sayıları aynı "matematiksel düzleme" getirir
from sklearn.preprocessing import StandardScaler

# Ölçeklenecek sütunları seç
scale_cols = [c for c in ['charge_count','arrest_hour_of_day'] if c in df.columns]

# Ölçekleyiciyi başlat
scaler = StandardScaler()

# Dönüşümü Uygula (Z-Skor Normalizasyonu)
# Formül: (Değer - Ortalama) / Standart Sapma
if scale_cols:
    df[scale_cols] = scaler.fit_transform(df[scale_cols])

print("✔ StandardScaler uygulandı.")


✔ StandardScaler uygulandı.


In [13]:

print("\n--- FEATURE SELECTION (PCA Dimension Reduction) ---")

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer # EKLENDİ: Boşluk doldurucu

# 1) Hedef ve Özellik Ayrımı
if 'is_onview_arrest' in df.columns:
    X = df.drop(columns=['is_onview_arrest'])
    Y = df['is_onview_arrest']
else:
    # Eğer yanlışlıkla silindiyse, sütunu tekrar oluşturmayı dene
    if 'arrest_type' in df.columns:
        print("⚠ Hedef değişken yeniden oluşturuluyor...")
        Y = np.where(df['arrest_type']=='O', 1, 0)
        X = df.drop(columns=['arrest_type', 'arrest_translation'], errors='ignore')
    else:
        raise KeyError("HATA: Hedef değişken bulunamadı. Lütfen en baştan tekrar çalıştırın.")

# 2) Sayısal verileri seçme
X_numeric = X.select_dtypes(include=['int64', 'float64', 'int32', 'float32'])

# PCA öncesi veride milimetrik boşluklar kaldıysa onları temizliyoruz.
if X_numeric.isnull().values.any():
    print(f"⚠ Uyarı: Veride {X_numeric.isnull().sum().sum()} adet boş değer tespit edildi.")
    print("✔ PCA öncesi 'SimpleImputer' ile boşluklar ortalama değerle dolduruluyor...")
    
    imputer = SimpleImputer(strategy='mean')
    # Sütun isimlerini kaybetmemek için işlem sonrası tekrar DataFrame yapıyoruz
    cols = X_numeric.columns
    X_numeric = pd.DataFrame(imputer.fit_transform(X_numeric), columns=cols)

# 3) Ölçeklendirme (StandardScaler)
scaler_pca = StandardScaler()
X_scaled = scaler_pca.fit_transform(X_numeric)

# 4) PCA Uygulama
pca = PCA(n_components=0.95)  
X_pca = pca.fit_transform(X_scaled)

print(f"✔ PCA tamamlandı. Yeni bileşen sayısı: {X_pca.shape[1]}")

# 5) Sonuçları DataFrame’e çevir
pca_cols = [f"PCA_{i+1}" for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(X_pca, columns=pca_cols)

# Final feature subset 
X_final = X_pca_df.copy()
y_final = Y.copy()

print("\n✔ İŞLEM BAŞARILI: PCA tabanlı feature matrix hazır.")
display(X_final.head())


--- FEATURE SELECTION (PCA Dimension Reduction) ---
✔ PCA tamamlandı. Yeni bileşen sayısı: 75

✔ İŞLEM BAŞARILI: PCA tabanlı feature matrix hazır.


,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,...,PCA_66,PCA_67,PCA_68,PCA_69,PCA_70,PCA_71,PCA_72,PCA_73,PCA_74,PCA_75
0,-2.630361,1.952524,-1.728320,2.669809,-0.703561,-1.217302,-0.620678,0.736996,-0.081185,0.402873,...,-0.157709,-0.184736,-0.156251,-0.259274,0.156894,0.999244,-1.195155,0.245522,0.972361,0.209064
1,-0.700716,-0.344666,3.516237,0.762764,0.176024,0.086914,-2.773570,1.467302,0.515845,0.652864,...,0.032020,0.069272,0.687735,-0.192730,-0.046207,-0.491476,0.521788,-0.906214,0.382673,-0.013411
2,-0.156300,-2.851432,0.677287,-0.788573,-1.802286,0.047755,-2.907700,0.699595,0.644415,0.225007,...,-0.117033,-0.273458,-0.336374,0.064567,0.004859,-0.126432,0.937545,-1.132038,-0.029192,0.298584
3,1.218483,3.234061,-0.932658,0.613361,-1.010378,0.400309,-0.728549,0.994098,0.221375,0.488268,...,0.186686,-0.260760,-0.110496,-0.584038,0.167301,0.527076,-1.156078,0.295594,-0.541520,-1.464257
4,-0.451422,-2.970364,-0.597476,-4.232937,-1.639593,-3.817581,0.112978,-0.103911,0.820725,-0.597111,...,0.598567,0.648612,-0.810627,1.672977,-0.786192,1.071030,-0.515134,0.527539,-0.604165,2.200816


In [14]:
#PCA'dan gelen özellikler (X) ve hedef değişkeni (Y) birleştir

df_final = pd.concat([X_final, y_final], axis=1)

print("\n✔ Final dataset hazır.")
print(f"Boyut: {df_final.shape}")
display(df_final.head())



✔ Final dataset hazır.
Boyut: (47444, 76)


,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,...,PCA_67,PCA_68,PCA_69,PCA_70,PCA_71,PCA_72,PCA_73,PCA_74,PCA_75,is_onview_arrest
0,-2.630361,1.952524,-1.728320,2.669809,-0.703561,-1.217302,-0.620678,0.736996,-0.081185,0.402873,...,-0.184736,-0.156251,-0.259274,0.156894,0.999244,-1.195155,0.245522,0.972361,0.209064,1
1,-0.700716,-0.344666,3.516237,0.762764,0.176024,0.086914,-2.773570,1.467302,0.515845,0.652864,...,0.069272,0.687735,-0.192730,-0.046207,-0.491476,0.521788,-0.906214,0.382673,-0.013411,0
2,-0.156300,-2.851432,0.677287,-0.788573,-1.802286,0.047755,-2.907700,0.699595,0.644415,0.225007,...,-0.273458,-0.336374,0.064567,0.004859,-0.126432,0.937545,-1.132038,-0.029192,0.298584,0
3,1.218483,3.234061,-0.932658,0.613361,-1.010378,0.400309,-0.728549,0.994098,0.221375,0.488268,...,-0.260760,-0.110496,-0.584038,0.167301,0.527076,-1.156078,0.295594,-0.541520,-1.464257,1
4,-0.451422,-2.970364,-0.597476,-4.232937,-1.639593,-3.817581,0.112978,-0.103911,0.820725,-0.597111,...,0.648612,-0.810627,1.672977,-0.786192,1.071030,-0.515134,0.527539,-0.604165,2.200816,0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier

# TRAIN-TEST SPLIT 
results = []

#PCA Öncesi 
X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, y_final, test_size=0.25, random_state=42, stratify=y_final
)

#PCA Sonrası
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
)
print("✔ Veri setleri eğitim ve test olarak ayrıldı.")


✔ Veri setleri eğitim ve test olarak ayrıldı.


In [16]:
# Metrik fonksiyonu

def evaluate_model(model,X_tr, X_te, y_tr, y_te, model_name):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    # ROC-AUC için olasılık veya karar fonksiyonu
    if hasattr(model, "predict_proba"):
        # Olasılık (Probability)
        y_prob = model.predict_proba(X_te)[:, 1]
    else:
        #Karar Fonksiyonu (Decision Function)
        dec = model.decision_function(X_te)
        # Min-Max Normalizasyonu ile 0-1 arasına çekme
        y_prob = (dec - dec.min()) / (dec.max() - dec.min() + 1e-9)
         
    # Metrikler
    acc = accuracy_score(y_te, y_pred)
    prec = precision_score(y_te, y_pred, zero_division=0)
    rec = recall_score(y_te, y_pred, zero_division=0)
    f1 = f1_score(y_te, y_pred, zero_division=0)
    roc = roc_auc_score(y_te, y_prob)

    print(f"\n>>> {model_name}")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print(f"ROC-AUC  : {roc:.4f}")

    return [acc, prec, rec, f1, roc]

print("✔ Fonksiyon tanımlandı.")

✔ Fonksiyon tanımlandı.


In [17]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

print("=== PCA ETKİ ANALİZİ VE MODELLER ===")

# --- 1. DUMB MODEL (Sadece PCA Öncesi / Ham Veri) ---
# Amaç: Veri setinin en doğal halindeki taban puanı görmek.
print("\n--- 1. Dumb (Baseline) Model ---")
dummy_model = DummyClassifier(strategy='most_frequent', random_state=42)

# X_train ve X_test (PCA uygulanmamış ham veri) kullanıyoruz
res_dummy = evaluate_model(dummy_model, X_train, X_test, y_train, y_test, "Dumb (Ham Veri)")
results.append(["Raw + Dummy", *res_dummy])


# --- 2. LOGISTIC REGRESSION (Karşılaştırmalı) ---
print("\n--- 2. Logistic Regression ---")
log_reg = LogisticRegression(max_iter=10000, solver='liblinear', random_state=42)

# A) PCA ÖNCESİ (Ham Veri)
res_log_raw = evaluate_model(log_reg, X_train, X_test, y_train, y_test, "LogReg")
results.append(["Raw + LogReg", *res_log_raw])

# B) PCA SONRASI
res_log_pca = evaluate_model(log_reg, X_train_pca, X_test_pca, y_train_pca, y_test_pca, "LogReg (PCA)")
results.append(["PCA + LogReg", *res_log_pca])


# --- 3. RIDGE CLASSIFIER (Karşılaştırmalı) ---
print("\n--- 3. Ridge Classifier ---")
ridge_model = RidgeClassifier(random_state=42)

# A) PCA ÖNCESİ (Ham Veri)
res_ridge_raw = evaluate_model(ridge_model, X_train, X_test, y_train, y_test, "Ridge")
results.append(["Raw + Ridge", *res_ridge_raw])

# B) PCA SONRASI
res_ridge_pca = evaluate_model(ridge_model, X_train_pca, X_test_pca, y_train_pca, y_test_pca, "Ridge (PCA)")
results.append(["PCA + Ridge", *res_ridge_pca])

print("\n✔ Tüm modeller tamamlandı. PCA etkisi tabloda görülebilir.")

=== PCA ETKİ ANALİZİ VE MODELLER ===

--- 1. Dumb (Baseline) Model ---

>>> Dumb (Ham Veri)
Accuracy : 0.7585
Precision: 0.0000
Recall   : 0.0000
F1 Score : 0.0000
ROC-AUC  : 0.5000

--- 2. Logistic Regression ---

>>> LogReg
Accuracy : 0.7585
Precision: 0.0000
Recall   : 0.0000
F1 Score : 0.0000
ROC-AUC  : 0.4579

>>> LogReg (PCA)
Accuracy : 0.7644
Precision: 0.5745
Recall   : 0.0942
F1 Score : 0.1619
ROC-AUC  : 0.7258

--- 3. Ridge Classifier ---

>>> Ridge
Accuracy : 0.7638
Precision: 0.5920
Recall   : 0.0719
F1 Score : 0.1282
ROC-AUC  : 0.7282

>>> Ridge (PCA)
Accuracy : 0.7633
Precision: 0.5893
Recall   : 0.0656
F1 Score : 0.1181
ROC-AUC  : 0.7224

✔ Tüm modeller tamamlandı. PCA etkisi tabloda görülebilir.
